In [ ]:
import os
from pathlib import Path

import crystal_toolkit  # noqa: F401
from pymatgen.core import Structure

In [52]:
# batch_name = "bandgap3.5b1"
batch_name = "ferrite"
batch_path = "../output"
cif_name = "2_POSCAR-0.25"
cif_path = os.path.abspath(os.path.join(batch_path, batch_name, "extracted", f"{cif_name}"))

print(f"Loading {cif_path}")

struct_expt = Structure.from_file(cif_path)
print(struct_expt)
struct_expt

In [54]:
# get reduced formula
reduced_formula = struct_expt.composition.reduced_formula
print(f"Reduced formula: {reduced_formula}")

output_dir = f"{batch_name}-{cif_name}-{reduced_formula}"

# mkdir in output folder named reduced_formula
os.makedirs(f"output/{output_dir}", exist_ok=True)

Reduced formula: Mn3Zn(FeO2)8


## pbe

In [ ]:
# # Relax with pbe

# import matcalc

# calculator_pbe = matcalc.load_fp("pbe")
# relax_calc = matcalc.RelaxCalc( # type: ignore
#     calculator_pbe,
#     optimizer="FIRE",
#     relax_atoms=True,
#     relax_cell=True
# )

# data = relax_calc.calc(struct_expt)

# final_structure_pbe = data['final_structure']
# print(final_structure_pbe)
# final_structure_pbe

# # Save the final structure
# final_structure_pbe.to(fmt="poscar", filename=f"output/{output_dir}/POSCAR_pbe")

In [ ]:
# phonon_calc = matcalc.PhononCalc( # type: ignore
#     calculator_pbe,
#     relax_structure=True,
#     write_band_structure=f"output/{output_dir}/pbe_phonon_bs.yaml",
#     write_total_dos=f"output/{output_dir}/pbe_phonon_dos.dat",
#     write_phonon=f"output/{output_dir}/pbe_phonon.yaml",
#     optimizer="FIRE",
# )

# data_phonon_pbe = phonon_calc.calc(final_structure_pbe)

In [ ]:
# import matplotlib.pyplot as plt

# phonon_bs = data_phonon_pbe['phonon'].band_structure

# n_axes = sum(1 for c in phonon_bs.path_connections if not c)

# fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))

# if n_axes == 1:
#     axs = [axs]

# for ax in axs[1:]:
#     ax.set_ylabel('')
#     ax.tick_params(axis='y', labelleft=False)

# phonon_bs.plot(ax=axs)
# fig.suptitle("Phonon band structure", fontsize=16)
# plt.subplots_adjust(wspace=0.07)
# plt.show()


In [ ]:
# phonon_dos = data_phonon_pbe["phonon"].total_dos

# fig, ax = plt.subplots(figsize=(8, 5))
# phonon_dos.plot(ax=ax)
# fig.suptitle("Phonon DOS", fontsize=16)
# plt.show()

In [ ]:
# plt.plot(
#     data_phonon_pbe['thermal_properties']['temperatures'],
#     data_phonon_pbe['thermal_properties']['heat_capacity']
# )

# plt.xlabel("Temperature (K)")
# plt.ylabel('Heat Capacity (J/mol-K)')
# plt.title("Heat Capacity vs Temperature")
# plt.show()

## r2scan

In [ ]:
## Relax with r2scan

import matcalc

calculator_r2scan = matcalc.load_fp("r2scan")
relax_calc = matcalc.RelaxCalc(
    calculator_r2scan,
    optimizer="FIRE",
    relax_atoms=True,
    relax_cell=True
)

data_r2scan = relax_calc.calc(struct_expt)


In [ ]:
final_structure_r2scan = data_r2scan['final_structure']
print(final_structure_r2scan)
final_structure_r2scan

# Save the final structure
final_structure_r2scan.to(fmt="poscar", filename=f"output/{output_dir}/POSCAR_r2scan")

In [ ]:
phonon_calc = matcalc.PhononCalc( # type: ignore
    calculator_r2scan,
    relax_structure=True,
    write_band_structure=f"output/{output_dir}/r2scan_phonon_bs.yaml",
    write_total_dos=f"output/{output_dir}/r2scan_phonon_dos.dat",
    write_phonon=f"output/{output_dir}/r2scan_phonon.yaml",
    optimizer="FIRE",
)

data_phonon_r2scan = phonon_calc.calc(final_structure_r2scan)

In [ ]:
import matplotlib.pyplot as plt

phonon_bs = data_phonon_r2scan['phonon'].band_structure

n_axes = sum(1 for c in phonon_bs.path_connections if not c)

fig, axs = plt.subplots(1, n_axes, figsize=(16, 5))

if n_axes == 1:
    axs = [axs]

for ax in axs[1:]:
    ax.set_ylabel('')
    ax.tick_params(axis='y', labelleft=False)

phonon_bs.plot(ax=axs)
fig.suptitle("Phonon band structure", fontsize=16)
plt.subplots_adjust(wspace=0.07)
plt.show()


In [ ]:
phonon_dos = data_phonon_r2scan["phonon"].total_dos

fig, ax = plt.subplots(figsize=(8, 5))
phonon_dos.plot(ax=ax)
fig.suptitle("Phonon DOS", fontsize=16)
plt.show()

In [ ]:
plt.plot(
    data_phonon_r2scan['thermal_properties']['temperatures'],
    data_phonon_r2scan['thermal_properties']['heat_capacity']
)

plt.xlabel("Temperature (K)")
plt.ylabel('Heat Capacity (J/mol-K)')
plt.title("Heat Capacity vs Temperature")
plt.show()

In [ ]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core import Structure

def plot_phonon_band(band_yaml_file, structure_file):
    """
    Plot phonon dispersion directly from band.yaml (even if 'lattice_rec' is missing).
    Imaginary frequencies are shown in red.
    """
    try:
        structure = Structure.from_file(structure_file)
        with open(band_yaml_file, "r") as f:
            data = yaml.safe_load(f)

        q_labels = [p["label"] if "label" in p else "" for p in data.get("labels", [])]
        q_distances = []
        frequencies = []

        # Extract phonon data
        for point in data["phonon"]:
            q_distances.append(point["distance"])
            freqs = [band["frequency"] for band in point["band"]]
            frequencies.append(freqs)

        q_distances = np.array(q_distances)
        frequencies = np.array(frequencies)

        # Plot
        plt.figure(figsize=(10, 6))
        for i in range(frequencies.shape[1]):
            freq_branch = frequencies[:, i]
            plt.plot(q_distances, freq_branch,
                     color="red" if np.any(freq_branch < 0) else "black", lw=1.2)

        plt.axhline(0, color="gray", lw=0.8)
        plt.xlabel("Wave vector path")
        plt.ylabel("Frequency (THz)")
        plt.title("Phonon Band Structure")
        plt.grid(alpha=0.2)
        plt.show()

        min_freq = np.min(frequencies)
        if min_freq < 0:
            print(f"❌ Imaginary frequencies detected (min = {min_freq:.3f} THz). Material unstable.")
        else:
            print(f"✅ All frequencies positive (min = {min_freq:.3f} THz). Material stable.")

    except Exception as e:
        print(f"⚠️ Error while plotting phonon band: {e}")

band_file = f"output/{output_dir}/r2scan_phonon_bs.yaml"
structure_file = f"output/{output_dir}/POSCAR_r2scan"

plot_phonon_band(band_file, structure_file)